### Simple Full Archive Search

In [1]:
import requests
import os
import json
import time
import datetime
import dateutil.parser
import osmnx as ox
from shapely.geometry import box

In [17]:
# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
file_name_base = '../data/' #enter path where you want the file saved to

bearer_token = "AAAAAAAAAAAAAAAAAAAAAI1PNgEAAAAATarF1xcbxxXdqM15JyHfxL8TLqk%3DTbEmtm5q9UZ0rDTUQa0wUerv9w3LHZAX3UM9gn3qDNvFHhTB56"#os.environ.get("BEARER_TOKEN")

search_url = "https://api.twitter.com/2/tweets/search/all"

# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
# query_params = {'query': '(from:twitterdev -is:retweet) OR #twitterdev','tweet.fields': 'author_id'}
query_params = {'query': f'(("george floyd" OR "elijah mcclain" OR "breonna taylor" OR (police brutality) OR blacklivesmatter OR (police violence) OR blm OR (racism police) OR (blacklives) OR (police accountability) OR "police reform" OR (defund police) OR (abolish police) OR "abolish the police" OR "police abolition" OR (police racist) OR (brutal police) OR (black lives) OR (police kill black) OR (black killed police) OR ("no justice" "no peace") OR "murdered by police" OR (police profiling) OR (police discrimination)) OR (#policereform OR #policeaccountability OR #defundthepolice OR #abolishthepolice OR #abolishpolice OR #defundpolice OR #criminaljusticereform OR #notoracism OR #blacklivesmatter OR #blm OR #georgefloyd OR #justiceforgeorgefloyd OR #icantbreathe OR #nojusticenopeace OR #justiceforfloyd OR #justiceforgeorge OR #sayhername OR #stopracismnow OR #endracism OR #blacklivesstillmatter OR #taketheknee OR #breonnataylor OR #justiceforbreonnataylor OR #blackwomenmatter)) has:geo point_radius:[-73.935242 40.730610 25mi]',
#                 'tweet.fields': 'author_id,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,possibly_sensitive,public_metrics,referenced_tweets,reply_settings,source,text,withheld', point_radius:[-73.935242 40.730610 25mi] "tamir rice" OR "trayvon martin" OR "elijah mcclain" OR "breonna taylor" OR "emmett till" OR "mike brown" #emmettill
                'tweet.fields': 'author_id,conversation_id,created_at,geo,entities,id,in_reply_to_user_id,referenced_tweets,public_metrics,text',
                'place.fields': 'contained_within,country,country_code,full_name,geo,id,name,place_type',
                'max_results': '10',
                'expansions': 'geo.place_id',#,author_id,entities.mentions.username,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id',
#                 'user.fields': 'created_at,description,id,location,name,protected,public_metrics,url,username,verified',
                'start_time': '2020-05-20T00:00:00.00Z',
#                     'end_time': '2020-05-27T00:00:00.00Z'}
                'end_time': '2020-11-10T00:00:00.00Z'}


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers, params):
    response = requests.request("GET", search_url, headers=headers, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def main():
    headers = create_headers(bearer_token)
    json_response = connect_to_endpoint(search_url, headers, query_params)
    print(json.dumps(json_response, indent=4, sort_keys=True))
    ts = json_response['data'][-1]['created_at']
    stamp = dateutil.parser.parse(ts).strftime('%m%d%H%M%S_%y')
    # save one file for each response
    file_name = file_name_base + stamp + '.json'
    with open(file_name , 'w') as f:
        json.dump(json_response, f)

    print("Collected", len(json_response['data']), "tweets from", json_response['data'][0]['created_at'], "to", ts)
    # update query with "next_token" from the most recent tweet to go to the next search page
    query_params['next_token'] = json_response['meta']['next_token']
    
#     return json_response

# if __name__ == "__main__":
# #     test = main()
#     main()

    
if __name__ == "__main__":
    while(True):
        try:
            main()  
        except:
            time.sleep(10)
            print("program restart")

### Full archive search with more detailed geo-information

In [2]:
# Creating an H3 Hexagon Grid
import pandas as pd
import os
import numpy as np
import itertools
from pprint import pprint
from h3 import h3
import folium
import shapely.geometry
import geopandas as gpd
import requests, zipfile, io
from datetime import datetime

def create_hexgrid(polygon, hex_res, geometry_col='geometry', map_zoom=12, buffer=0.000,
                   stroke_weight=0.5, stroke_color='blue', plot=True):
    """ Takes in a geopandas geodataframe, the desired resolution, the specified geometry column
        and some map parameters to create a hexagon grid (and potentially plot the hexgrid
    """
    centroid = list(polygon.centroid.values[0].coords)[0]
    fol_map = folium.Map(location=[centroid[1], centroid[0]], zoom_start=map_zoom,
                         tiles='cartodbpositron')

    # Explode multipolygon into individual polygons
    exploded = polygon.explode().reset_index(drop=True)

    # Master lists for geodataframe
    hexagon_polygon_list = []
    hexagon_geohash_list = []

    # For each exploded polygon
    for poly in exploded[geometry_col].values:

        # Reverse coords for original polygon
        coords = list(poly.exterior.coords)
        reversed_coords = []
        for i in coords:
            reversed_coords.append([i[1], i[0]])

        # Reverse coords for buffered polygon
        buffer_poly = poly.buffer(buffer)
        buffer_coords = list(buffer_poly.exterior.coords)
        reversed_buffer_coords = []
        for i in buffer_coords:
            reversed_buffer_coords.append([i[1], i[0]])

        # Format input to the way H3 expects it
        aoi_input = {'type': 'Polygon', 'coordinates': [reversed_buffer_coords]}

        # Add polygon outline to map
        outline = reversed_coords
        outline.append(outline[0])
        outline = folium.PolyLine(locations=outline, weight=1, color='black')
        fol_map.add_child(outline)

        # Generate list geohashes filling the AOI
        geohashes = list(h3.polyfill(aoi_input, hex_res))

        # Generate hexagon polylines for Folium
        polylines = []
        lat = []
        lng = []
        for geohash in geohashes:
            polygons = h3.h3_set_to_multi_polygon([geohash], geo_json=False)
            outlines = [loop for polygon in polygons for loop in polygon]
            polyline = [outline + [outline[0]] for outline in outlines][0]
            lat.extend(map(lambda x: x[0], polyline))
            lng.extend(map(lambda x: x[1], polyline))
            polylines.append(polyline)
            hexagon_geohash_list.append(geohash)

        # Add the hexagon polylines to Folium map
        for polyline in polylines:
            my_polyline = folium.PolyLine(locations=polyline, weight=stroke_weight,
                                          color=stroke_color)
            fol_map.add_child(my_polyline)

        # Generate hexagon polygons for Shapely
        for geohash in geohashes:
            polygons = h3.h3_set_to_multi_polygon([geohash], geo_json=True)
            outlines = [loop for polygon in polygons for loop in polygon]
            polyline_geojson = [outline + [outline[0]] for outline in outlines][0]
            hexagon_polygon_list.append(shapely.geometry.Polygon(polyline_geojson))

    if plot:
        display(fol_map)

    # Create a geodataframe containing the hexagon geometries and hashes
    hexgrid_gdf = gpd.GeoDataFrame()
    hexgrid_gdf['geometry'] = hexagon_polygon_list
    id_col_name = 'hex_id_' + str(hex_res)
    hexgrid_gdf[id_col_name] = hexagon_geohash_list
    hexgrid_gdf.crs = {'init' :'epsg:4326'}

    # Drop duplicate geometries
    geoms_wkb = hexgrid_gdf["geometry"].apply(lambda geom: geom.wkb)
    hexgrid_gdf = hexgrid_gdf.loc[geoms_wkb.drop_duplicates().index]

    return hexgrid_gdf

In [3]:
import osmnx as ox
from shapely.geometry import box
# example of NYC
city = "United States"
# city = "Chicago"
city_AREA = ox.gdf_from_place(city)

# city_AREA = gpd.GeoDataFrame(ox.gdf_from_place(city), geometry=[box(*ox.gdf_from_place(city).bounds.values[0])])
res = 4 #https://github.com/uber/h3/blob/master/docs/core-library/restable.md
grid_city = create_hexgrid(city_AREA, hex_res = res, geometry_col='geometry', map_zoom=11, buffer=0.000,
                        stroke_weight=0.5, stroke_color='blue', plot=False)

In [9]:
grid_city.to_file(f"../data/grids/US_{res}res_grid.geojson", driver="GeoJSON")

In [10]:
# for index, row in grid_city.iloc[:10,:].iterrows():
#     # print(row.geometry.centroid.x, row.geometry.centroid.y)

#     radius = "17mi"
#     # change point radius query for each area
#     geo_query = f"{row.geometry.centroid.x} {row.geometry.centroid.y} {radius}"
    
#     hex_id = row[f"hex_id_{res}"]
    
#     query_params = {'query': f'(("george floyd" OR "elijah mcclain" OR "breonna taylor" OR (police brutality) OR blacklivesmatter OR (police violence) OR blm OR (racism police) OR (blacklives) OR (police accountability) OR "police reform" OR (defund police) OR (abolish police) OR "abolish the police" OR "police abolition" OR (police racist) OR (brutal police) OR (black lives) OR (police kill black) OR (black killed police) OR ("no justice" "no peace") OR "murdered by police" OR (police profiling) OR (police discrimination)) OR (#policereform OR #policeaccountability OR #defundthepolice OR #abolishthepolice OR #abolishpolice OR #defundpolice OR #criminaljusticereform OR #notoracism OR #blacklivesmatter OR #blm OR #georgefloyd OR #justiceforgeorgefloyd OR #icantbreathe OR #nojusticenopeace OR #justiceforfloyd OR #justiceforgeorge OR #sayhername OR #stopracismnow OR #endracism OR #blacklivesstillmatter OR #taketheknee OR #breonnataylor OR #justiceforbreonnataylor OR #blackwomenmatter)) has:geo point_radius:[{geo_query}]',
#                     'tweet.fields': 'author_id,conversation_id,created_at,geo,entities,id,in_reply_to_user_id,referenced_tweets,public_metrics,text',
#                     'place.fields': 'contained_within,country,country_code,full_name,geo,id,name,place_type',
#                     'max_results': '500',
#                     'expansions': 'geo.place_id,author_id,entities.mentions.username,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id',
#                     'user.fields': 'created_at,description,id,location,name,protected,public_metrics,url,username,verified',
#                     'start_time': '2020-05-20T00:00:00.00Z',
#                     'end_time': '2020-11-10T00:00:00.00Z'}
#     print(query_params)
#     print("_____")

### Loop through US-Hex-Grid (US Wide Collection)

In [19]:
%%time
# hex_NYC.geometry[100].centroid.y
import requests
import os
import json
import time
import datetime
import dateutil.parser

# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
# file_name_base = '../data/' #enter path where you want the file saved to
file_name_base = 'E:/police_brutality_tweets/full_history_geo/'

# bearer_token = "AAAAAAAAAAAAAAAAAAAAAI1PNgEAAAAATarF1xcbxxXdqM15JyHfxL8TLqk%3DTbEmtm5q9UZ0rDTUQa0wUerv9w3LHZAX3UM9gn3qDNvFHhTB56"#os.environ.get("BEARER_TOKEN")

bearer_token = "AAAAAAAAAAAAAAAAAAAAAMiRMwEAAAAAajrtTBAkLc2mbCdBNTt63PARNqg%3D8o4ZQfDKVvKX0JS1QqHxCb0ZweiOx8dFLt0BQKJJctzwTihPi5"#os.environ.get("BEARER_TOKEN")

search_url = "https://api.twitter.com/2/tweets/search/all"

# city = "test"
# set search radius in miles
# radius = "0.76mi"
# radius = "2.15mi"
radius = "17mi"
# iterate through each hexagon's centroid
for index, row in grid_city.iloc[1285:,:].iterrows():
    # print(row.geometry.centroid.x, row.geometry.centroid.y)

    # change point radius query for each area
    geo_query = f"{row.geometry.centroid.x} {row.geometry.centroid.y} {radius}"
    
    hex_id = row[f"hex_id_{res}"]
    print("Searching tweets within hexagon", hex_id, ": ", index, "out of", str(len(grid_city)), "hexagons scanned for tweets!")
    # print(geo_query)
    # update query at each iteration
#     query_params = {'query': f'(("george floyd" OR "tamir rice" OR "trayvon martin" OR "elijah mcclain" OR "breonna taylor" OR "breonna" OR "emmett till" OR "mike brown" OR "eric garner" OR "police brutality" OR "black lives matter" OR blacklivesmatter OR "police violence" OR blm OR racism OR "black lives" OR "police accountability" OR "police reform" OR "defund the police" OR "defund police" OR "abolish police" OR "abolish the police" OR "police abolition") OR (#policereform OR #policeaccountability OR #defundthepolice OR #abolishthepolice OR #abolishpolice OR #defundpolice OR #criminaljusticereform OR #notoracism OR #blacklivesmatter OR #blm OR #georgefloyd OR #justiceforgeorgefloyd OR #icantbreathe OR #nojusticenopeace OR #justiceforfloyd OR #justiceforgeorge OR #sayhername OR #stopracismnow OR #endracism OR #blacklivesstillmatter OR #blacktranslivesmatter OR #taketheknee OR #breonnataylor OR #justiceforbreonnataylor OR #blackwomenmatter OR #justiceforbre)) has:geo point_radius:[{geo_query}]',
    query_params = {'query': f'(("george floyd" OR "elijah mcclain" OR "breonna taylor" OR (police brutality) OR blacklivesmatter OR (police violence) OR blm OR (racism police) OR (blacklives) OR (police accountability) OR "police reform" OR (defund police) OR (abolish police) OR "abolish the police" OR "police abolition" OR (police racist) OR (brutal police) OR (black lives) OR (police kill black) OR (black killed police) OR ("no justice" "no peace") OR "murdered by police" OR (police profiling) OR (police discrimination)) OR (#policereform OR #policeaccountability OR #defundthepolice OR #abolishthepolice OR #abolishpolice OR #defundpolice OR #criminaljusticereform OR #notoracism OR #blacklivesmatter OR #blm OR #georgefloyd OR #justiceforgeorgefloyd OR #icantbreathe OR #nojusticenopeace OR #justiceforfloyd OR #justiceforgeorge OR #sayhername OR #stopracismnow OR #blacklivesstillmatter OR #taketheknee OR #breonnataylor OR #justiceforbreonnataylor OR #blackwomenmatter)) has:geo point_radius:[{geo_query}]',
                    'tweet.fields': 'author_id,conversation_id,created_at,geo,entities,id,in_reply_to_user_id,referenced_tweets,public_metrics,text',
                    'place.fields': 'contained_within,country,country_code,full_name,geo,id,name,place_type',
                    'max_results': '500',
                    'expansions': 'geo.place_id,author_id,entities.mentions.username,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id',
                    'user.fields': 'created_at,description,id,location,name,protected,public_metrics,url,username,verified',
                    'start_time': '2020-05-20T00:00:00.00Z',
                    'end_time': '2020-11-10T00:00:00.00Z'}


    def create_headers(bearer_token):
        headers = {"Authorization": "Bearer {}".format(bearer_token)}
        return headers


    def connect_to_endpoint(url, headers, params):
        response = requests.request("GET", search_url, headers=headers, params=params)
        print(response.status_code)
        if response.status_code != 200:
            raise Exception(response.status_code, response.text)
        return response.json()


    def main():
        headers = create_headers(bearer_token)
        json_response = connect_to_endpoint(search_url, headers, query_params)
#         print(json.dumps(json_response, indent=4, sort_keys=True))
        
        try:
            ts = json_response['data'][-1]['created_at']
            stamp = dateutil.parser.parse(ts).strftime('%m%d%H%M%S_%y')
            # save one file for each response
            file_name = file_name_base + str(res) + 'res' + '/' + city.replace(" ", "") + "_" + hex_id + "_" + str(res) + "res" + "_" + stamp + '.json'
#             print(file_name)

            with open(file_name , 'w') as f:
#                 json.dump(json_response['data'], f)
                json.dump(json_response, f)

            print("Collected", len(json_response['data']), "tweets from", json_response['data'][0]['created_at'], 
                  "to", ts, "inside hexagon", hex_id)
            # update query with "next_token" from the most recent tweet to go to the next search page
            query_params['next_token'] = json_response['meta']['next_token']
            
        except KeyError:
            print("Response returned no data")
        
        return json_response

    #     return json_response

#     if __name__ == "__main__":
#     #     test = main()
#         main()


    if __name__ == "__main__":
        while(True):
            try:
                response = main()  
                if ("next_token" not in response['meta']) or (response['meta']["result_count"] == 0):
                    print("No more data to collect for hexagon", hex_id)
                    break
            except:
                print("Too many requests, sleeping...")
                time.sleep(20)
                print("Program restart!")

Searching tweets within hexagon 84298ebffffffff :  1285 out of 5796 hexagons scanned for tweets!
200
Response returned no data
No more data to collect for hexagon 84298ebffffffff
Searching tweets within hexagon 8428f55ffffffff :  1286 out of 5796 hexagons scanned for tweets!
200
Collected 6 tweets from 2020-11-09T00:15:08.000Z to 2020-10-15T04:32:16.000Z inside hexagon 8428f55ffffffff
200
Collected 28 tweets from 2020-09-30T02:11:37.000Z to 2020-08-10T19:21:16.000Z inside hexagon 8428f55ffffffff
200
Collected 23 tweets from 2020-08-08T22:27:27.000Z to 2020-07-11T19:55:48.000Z inside hexagon 8428f55ffffffff
200
Collected 16 tweets from 2020-07-06T00:37:15.000Z to 2020-06-30T09:56:37.000Z inside hexagon 8428f55ffffffff
200
Collected 22 tweets from 2020-06-28T04:52:47.000Z to 2020-06-11T02:09:42.000Z inside hexagon 8428f55ffffffff
200
Collected 55 tweets from 2020-06-11T01:57:29.000Z to 2020-05-27T10:09:46.000Z inside hexagon 8428f55ffffffff
Response returned no data
No more data to colle

### Loop through multiple cities test

In [ ]:
import osmnx as ox
from shapely.geometry import box
import requests
import os
import json
import time
import datetime
import dateutil.parser

# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
file_name_base = '../data/' #enter path where you want the file saved to

bearer_token = "AAAAAAAAAAAAAAAAAAAAAI1PNgEAAAAATarF1xcbxxXdqM15JyHfxL8TLqk%3DTbEmtm5q9UZ0rDTUQa0wUerv9w3LHZAX3UM9gn3qDNvFHhTB56"#os.environ.get("BEARER_TOKEN")

search_url = "https://api.twitter.com/2/tweets/search/all"

radius = "0.77mi" #"0.77mi" #2.15mi

res = 7 #https://github.com/uber/h3/blob/master/docs/core-library/restable.md


cities = ["Chicago", "New York City", "Houston", "Miami"]

for city in cities:
    # extract bounding box of each city
    city_AREA = gpd.GeoDataFrame(city_AREA, geometry=[box(*ox.gdf_from_place(city).bounds.values[0])])
    grid_city = create_hexgrid(city_AREA, hex_res = res, geometry_col='geometry', map_zoom=11, buffer=0.000,
                            stroke_weight=0.5, stroke_color='blue', plot=False)
    
    print(city, "has", str(len(grid_city)), "hexagons each with a radius of", radius)

    # iterate through each hexagon's centroid
    for index, row in grid_city.iterrows():
        # change point radius query for each area
        geo_query = f"{row.geometry.centroid.x} {row.geometry.centroid.y} {radius}"

        hex_id = row[f"hex_id_{res}"]
        print("Searching tweets within hexagon", hex_id, "of", city)
        # print(geo_query)
        # update query at each iteration
        query_params = {'query': f'(("george floyd" OR "tamir rice" OR "trayvon martin" OR "elijah mcclain" OR "breonna taylor" OR "breonna" OR "emmett till" OR "mike brown" OR "eric garner" OR "police brutality" OR "black lives matter" OR blacklivesmatter OR "police violence" OR blm OR racism OR "black lives" OR "police accountability" OR "police reform" OR "defund the police" OR "defund police" OR "abolish police" OR "abolish the police" OR "police abolition") OR (#policereform OR #policeaccountability OR #defundthepolice OR #abolishthepolice OR #abolishpolice OR #defundpolice OR #criminaljusticereform OR #notoracism OR #blacklivesmatter OR #blm OR #georgefloyd OR #justiceforgeorgefloyd OR #icantbreathe OR #nojusticenopeace OR #justiceforfloyd OR #justiceforgeorge OR #sayhername OR #stopracismnow OR #endracism OR #blacklivesstillmatter OR #blacktranslivesmatter OR #taketheknee OR #breonnataylor OR #justiceforbreonnataylor OR #blackwomenmatter OR #justiceforbre)) has:geo point_radius:[{geo_query}]',
        #                 'tweet.fields': 'author_id,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,possibly_sensitive,public_metrics,referenced_tweets,reply_settings,source,text,withheld', point_radius:[-73.935242 40.730610 25mi] "tamir rice" OR "trayvon martin" OR "elijah mcclain" OR "breonna taylor" OR "emmett till" OR "mike brown" #emmettill
                        'tweet.fields': 'author_id,conversation_id,created_at,geo,entities,id,in_reply_to_user_id,referenced_tweets,public_metrics,text',
                        'place.fields': 'contained_within,country,country_code,full_name,geo,id,name,place_type',
                        'max_results': '500',
                        'expansions': 'geo.place_id,author_id,entities.mentions.username,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id',
                        'user.fields': 'created_at,description,id,location,name,protected,public_metrics,url,username,verified',
                        'start_time': '2020-05-20T00:00:00.00Z',
    #                     'end_time': '2020-05-27T00:00:00.00Z'}
                        'end_time': '2020-11-10T00:00:00.00Z'}


        def create_headers(bearer_token):
            headers = {"Authorization": "Bearer {}".format(bearer_token)}
            return headers


        def connect_to_endpoint(url, headers, params):
            response = requests.request("GET", search_url, headers=headers, params=params)
            print(response.status_code)
            if response.status_code != 200:
                raise Exception(response.status_code, response.text)
            return response.json()


        def main():
            headers = create_headers(bearer_token)
            json_response = connect_to_endpoint(search_url, headers, query_params)
    #         print(json.dumps(json_response, indent=4, sort_keys=True))

            try:
                ts = json_response['data'][-1]['created_at']
                stamp = dateutil.parser.parse(ts).strftime('%m%d%H%M%S_%y')
                # save one file for each response
                file_name = file_name_base + str(res) + 'res' + '/' + city + "_" + hex_id + "_" + str(res) + "res" + "_" + stamp + '.json'
    #             print(file_name)

                with open(file_name , 'w') as f:
    #                 json.dump(json_response['data'], f)
                    json.dump(json_response, f)

                print("Collected", len(json_response['data']), "tweets from", json_response['data'][0]['created_at'], 
                      "to", ts, "inside hexagon", hex_id)
                # update query with "next_token" from the most recent tweet to go to the next search page
                query_params['next_token'] = json_response['meta']['next_token']

            except KeyError:
                print("Response returned no data")

            return json_response


        if __name__ == "__main__":
            while(True):
                try:
                    response = main()  
                    if "next_token" not in response['meta']:
                        print("No more data to collect for hexagon", hex_id)
                        break
                except:
                    print("Too many requests, sleeping...")
                    time.sleep(20)
                    print("Program restart!")

In [135]:
# import requests
# import os
# import json
# import time
# import datetime
# import dateutil.parser

# # To set your environment variables in your terminal run the following line:
# # export 'BEARER_TOKEN'='<your_bearer_token>'
# file_name_base = '../data/' #enter path where you want the file saved to

# bearer_token = "AAAAAAAAAAAAAAAAAAAAAI1PNgEAAAAATarF1xcbxxXdqM15JyHfxL8TLqk%3DTbEmtm5q9UZ0rDTUQa0wUerv9w3LHZAX3UM9gn3qDNvFHhTB56"#os.environ.get("BEARER_TOKEN")

# search_url = "https://api.twitter.com/2/tweets/search/all"

# city = "test"

# # set search radius in miles
# radius = "0.76mi"
# # radius = "2mi"
# # iterate through each hexagon's centroid
# for index, row in hex_NYC.iterrows():
#     # print(row.geometry.centroid.x, row.geometry.centroid.y)

#     # change point radius query for each area
#     geo_query = f"{row.geometry.centroid.x} {row.geometry.centroid.y} {radius}"
    
#     hex_id = row[f"hex_id_{res}"]
#     print("Searching tweets within hexagon", hex_id)
#     # print(geo_query)
#     # update query at each iteration 
#     # f'("george floyd" OR "police brutality" OR "black lives matter" OR blacklivesmatter OR "police violence" OR blm OR racism OR "black lives" OR "police accountability" OR "police reform" OR "defund the police" OR "defund police" OR "abolish police" OR "abolish the police" OR "police abolition") OR (#policereform OR #policeaccountability OR #defundthepolice OR #abolishthepolice OR #abolishpolice OR #defundpolice OR #criminaljusticereform OR #notoracism OR #blacklivesmatter OR #blm OR #georgefloyd OR #justiceforgeorgefloyd OR #icantbreathe OR #nojusticenopeace OR #justiceforfloyd OR #justiceforgeorge OR #sayhername OR #stopracismnow OR #endracism OR #blacklivesstillmatter OR #blacktranslivesmatter OR #taketheknee OR #breonnataylor OR #justiceforbreonnataylor OR #blackwomenmatter OR #justiceforbre) has:geo point_radius:[{geo_query}]',
#     query_params = {'query': f'(("george floyd" OR "police brutality" OR "black lives matter" OR blacklivesmatter OR "police violence" OR blm OR racism OR "black lives" OR "police accountability" OR "police reform" OR "defund the police" OR "defund police" OR "abolish police" OR "abolish the police" OR "police abolition") OR (#policereform OR #policeaccountability OR #defundthepolice OR #abolishthepolice OR #abolishpolice OR #defundpolice OR #criminaljusticereform OR #notoracism OR #blacklivesmatter OR #blm OR #georgefloyd OR #justiceforgeorgefloyd OR #icantbreathe OR #nojusticenopeace OR #justiceforfloyd OR #justiceforgeorge OR #sayhername OR #stopracismnow OR #endracism OR #blacklivesstillmatter OR #blacktranslivesmatter OR #taketheknee OR #breonnataylor OR #justiceforbreonnataylor OR #blackwomenmatter OR #justiceforbre)) has:geo point_radius:[{geo_query}]',
#     #                 'tweet.fields': 'author_id,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,possibly_sensitive,public_metrics,referenced_tweets,reply_settings,source,text,withheld', point_radius:[-73.935242 40.730610 25mi]
#                     'tweet.fields': 'author_id,conversation_id,created_at,geo,id,in_reply_to_user_id,referenced_tweets',
#                     'place.fields': 'contained_within,country,country_code,full_name,geo,id,name,place_type',
#                     'max_results': '500',
#                     'expansions': 'geo.place_id',
#     #                 'user.fields': 'created_at,description,id,location,name,protected,public_metrics,url,username,verified',
#                     'start_time': '2020-05-20T00:00:00.00Z',
# #                     'end_time': '2020-05-27T00:00:00.00Z'}
#                     'end_time': '2020-11-10T00:00:00.00Z'}


#     def create_headers(bearer_token):
#         headers = {"Authorization": "Bearer {}".format(bearer_token)}
#         return headers


#     def connect_to_endpoint(url, headers, params):
#         response = requests.request("GET", search_url, headers=headers, params=params)
#         print(response.status_code)
#         if response.status_code != 200:
#             raise Exception(response.status_code, response.text)
#         return response.json()


#     def main():
#         headers = create_headers(bearer_token)
#         json_response = connect_to_endpoint(search_url, headers, query_params)
# #         print(json.dumps(json_response, indent=4, sort_keys=True))
        
#         try:
#             ts = json_response['data'][-1]['created_at']
#             stamp = dateutil.parser.parse(ts).strftime('%m%d%H%M%S_%y')
#             # save one file for each response
#             file_name = file_name_base + city + "_" + hex_id + "_" + str(res) + "res" + "_" + stamp + '.json'
# #             print(file_name)

#             with open(file_name , 'w') as f:
# #                 json.dump(json_response['data'], f)
#                 json.dump(json_response, f)

#             print("Collected", len(json_response['data']), "tweets from", json_response['data'][0]['created_at'], 
#                   "to", ts, "inside hexagon", hex_id)
#             # update query with "next_token" from the most recent tweet to go to the next search page
#             query_params['next_token'] = json_response['meta']['next_token']
            
#         except KeyError:
#             print("Response returned no data")
        
#         return json_response

#     #     return json_response

# #     if __name__ == "__main__":
# #     #     test = main()
# #         main()


#     if __name__ == "__main__":
#         while(True):
#             try:
#                 response = main()  
#                 if "next_token" not in response['meta']:
#                     print("No more data to collect for hexagon", hex_id)
#                     break
#             except:
#                 print("Too many requests, sleeping...")
#                 time.sleep(20)
#                 print("Program restart!")